# Database (SQL Integration)
Load enhanced data from CSV, create SQLite DB (`stock_data.db`), insert data into `stock_prices` table. Includes schema for features like Daily_Return, SMAs, and optional Volatility_20.

In [1]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, text
from pathlib import Path

# Load enhanced CSV
enhanced_csv = 'enhanced_stock_data.csv'
if not Path(enhanced_csv).exists():
    raise FileNotFoundError(f"{enhanced_csv} not found. Run data_preprocessing_feature_engineering.ipynb first.")
combined_df = pd.read_csv(enhanced_csv)
combined_df['Date'] = pd.to_datetime(combined_df['Date'])
print(f"Loaded enhanced data: {combined_df.shape}")
print(f"Available columns: {combined_df.columns.tolist()}")

Loaded enhanced data: (11130, 12)
Available columns: ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Daily Return', 'SMA_20', 'SMA_50', 'Volume Change']


In [2]:
# Create SQLite engine (file-based for persistence)
engine = create_engine('sqlite:///stock_data.db', echo=False)  # echo=True for SQL logs if debugging

# Define table schema (matching combined_df columns; includes optional Volatility_20)
create_table_sql = """
CREATE TABLE IF NOT EXISTS stock_prices (
    Date DATE,
    Ticker TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Adj_Close REAL,
    Volume REAL,
    Daily_Return REAL,
    SMA_20 REAL,
    SMA_50 REAL,
    Volume_Change REAL,
    Volatility_20 REAL  -- Optional; will be NULL if not in data
);
"""

# Execute the create table (wrapped in text() for SQLAlchemy 2.0+ compatibility)
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

print("✓ Database 'stock_data.db' created and table 'stock_prices' ready.")

✓ Database 'stock_data.db' created and table 'stock_prices' ready.


In [3]:
# Clean data for insertion (handle NaNs, rename columns for SQL compatibility)
combined_df_clean = combined_df.copy()

# Drop rows with missing core columns (Date, Ticker)
combined_df_clean = combined_df_clean.dropna(subset=['Date', 'Ticker'])

# Fill NaNs in numerical columns with 0 (or use ffill/bfill if preferred)
numerical_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Daily Return', 'SMA_20', 'SMA_50', 'Volume Change']
for col in numerical_cols:
    if col in combined_df_clean.columns:
        combined_df_clean[col] = combined_df_clean[col].fillna(0)

# Add Volatility_20 if missing (set to NULL/0; compute later if needed)
if 'Volatility_20' not in combined_df_clean.columns:
    combined_df_clean['Volatility_20'] = 0  # Placeholder

# Format Date as YYYY-MM-DD string for SQL
combined_df_clean['Date'] = pd.to_datetime(combined_df_clean['Date']).dt.strftime('%Y-%m-%d')

# Rename columns (no spaces for SQL)
combined_df_clean = combined_df_clean.rename(columns={
    'Adj Close': 'Adj_Close',
    'Daily Return': 'Daily_Return',
    'Volume Change': 'Volume_Change'
})

# Reorder to match schema
cols_order = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume', 
              'Daily_Return', 'SMA_20', 'SMA_50', 'Volume_Change', 'Volatility_20']
combined_df_clean = combined_df_clean[cols_order]

# Insert data using to_sql (handles DataFrame to SQL; 'replace' overwrites if table exists)
combined_df_clean.to_sql('stock_prices', engine, if_exists='replace', index=False, method='multi')

print(f"✓ Data insertion complete: {len(combined_df_clean)} rows inserted.")

✓ Data insertion complete: 11130 rows inserted.


In [4]:
# Verify insertion with queries
with engine.connect() as conn:
    # Total row count
    row_count = conn.execute(text("SELECT COUNT(*) FROM stock_prices")).scalar()
    print(f"Total rows in table: {row_count}")
    
    # Preview first 5 rows
    preview_sql = "SELECT * FROM stock_prices LIMIT 5"
    preview_df = pd.read_sql(preview_sql, engine)
    print("\nPreview of inserted data:")
    print(preview_df)
    
    # Per-ticker count
    ticker_count_sql = "SELECT Ticker, COUNT(*) as Count FROM stock_prices GROUP BY Ticker"
    ticker_counts = pd.read_sql(ticker_count_sql, engine)
    print("\nRows per Ticker:")
    print(ticker_counts)

print("✓ SQL integration complete! Database ready for queries (e.g., add more cells for STDDEV fixes if needed).")

Total rows in table: 11130

Preview of inserted data:
         Date         Ticker         Open         High          Low  \
0  2022-09-26  ADANIPORTS.NS  3614.600098  3614.600098  3614.600098   
1  2022-09-27  ADANIPORTS.NS  3614.600098  3614.600098  3614.600098   
2  2022-09-28  ADANIPORTS.NS  3614.600098  3614.600098  3614.600098   
3  2022-09-29  ADANIPORTS.NS  3614.600098  3614.600098  3614.600098   
4  2022-09-30  ADANIPORTS.NS  3614.600098  3614.600098  3614.600098   

         Close    Adj_Close       Volume  Daily_Return       SMA_20  \
0  3614.600098  3614.600098  3614.600098           0.0  3614.600098   
1  3614.600098  3614.600098  3614.600098           0.0  3614.600098   
2  3614.600098  3614.600098  3614.600098           0.0  3614.600098   
3  3614.600098  3614.600098  3614.600098           0.0  3614.600098   
4  3614.600098  3614.600098  3614.600098           0.0  3614.600098   

        SMA_50  Volume_Change  Volatility_20  
0  3614.600098            0.0              0 

In [6]:
summary_sql = """
SELECT 
    AVG(Close) as Avg_Close,
    MIN(Close) as Min_Close,
    MAX(Close) as Max_Close,
    AVG(Daily_Return) as Avg_Daily_Return,
    AVG(Daily_Return * Daily_Return) as Avg_Squared_Return,
    AVG(Volume) as Avg_Volume,
    COUNT(*) as Total_Records
FROM stock_prices
WHERE Daily_Return IS NOT NULL;
"""

summary_df = pd.read_sql(summary_sql, engine)

# Calculate std manually in Python
summary_df['Std_Daily_Return'] = (summary_df['Avg_Squared_Return'] - summary_df['Avg_Daily_Return']**2)**0.5
summary_df.drop(columns='Avg_Squared_Return', inplace=True)

print("=== SQL Overall Summary ===")
print(summary_df)


=== SQL Overall Summary ===
     Avg_Close   Min_Close    Max_Close  Avg_Daily_Return   Avg_Volume  \
0  1854.494777  403.200012  3657.600098          0.169462  1854.494777   

   Total_Records  Std_Daily_Return  
0          11130          0.633173  


In [7]:
# Per-ticker summary (no STDDEV needed here)
ticker_summary_sql = """
SELECT 
    Ticker,
    COUNT(*) as Record_Count,
    AVG(Close) as Avg_Close,
    AVG(Daily_Return) as Avg_Return,
    MAX(Close) as Max_Close,
    MIN(Close) as Min_Close
FROM stock_prices
GROUP BY Ticker
ORDER BY Avg_Close DESC;
"""

ticker_summary = pd.read_sql(ticker_summary_sql, engine)
print("\n=== SQL Per-Ticker Summary ===")
print(ticker_summary)


=== SQL Per-Ticker Summary ===
           Ticker  Record_Count    Avg_Close  Avg_Return    Max_Close  \
0   TATAMOTORS.NS           742  3653.615933   -0.001972  3657.600098   
1   ADANIPORTS.NS           742  3611.671526   -0.000810  3614.600098   
2     RELIANCE.NS           742  3611.601715    2.524720  3614.600098   
3     HDFCBANK.NS           742  3059.562706    0.004206  3062.399902   
4         SBIN.NS           742  2519.974075    0.002520  2522.199951   
5           LT.NS           742  1941.016528    0.000471  3657.600098   
6    ICICIBANK.NS           742  1497.360917    0.000997  1497.500000   
7   HINDUNILVR.NS           742  1395.820130    0.000164  2522.199951   
8          TCS.NS           742  1392.054227    0.000533  3062.399902   
9   BAJFINANCE.NS           742  1170.604227    0.000735  1170.800049   
10         M&M.NS           742  1028.989353    0.004597  3614.600098   
11        INFY.NS           742   957.928179    0.000668  1497.500000   
12         ITC.NS  

In [11]:
threshold = overall_vol_df['Avg_Vol'][0] + 1 * ((overall_vol_df['Avg_Sq_Vol'][0] - overall_vol_df['Avg_Vol'][0]**2)**0.5)

high_vol_sql = f"""
SELECT Ticker, AVG(Volatility_20) as Avg_Volatility
FROM stock_prices
WHERE Volatility_20 IS NOT NULL
GROUP BY Ticker
HAVING Avg_Volatility > {threshold}
ORDER BY Avg_Volatility DESC
LIMIT 5;
"""

high_vol = pd.read_sql(high_vol_sql, engine)
print("\n=== High Volatility Tickers ===")
print(high_vol)



=== High Volatility Tickers ===
Empty DataFrame
Columns: [Ticker, Avg_Volatility]
Index: []


In [12]:
# Backup DB to CSV (for portability)
backup_sql = "SELECT * FROM stock_prices;"
backup_df = pd.read_sql(backup_sql, engine)
backup_df.to_csv('stock_data_backup.csv', index=False)
print("✓ DB backed up to 'stock_data_backup.csv'.")

✓ DB backed up to 'stock_data_backup.csv'.
